### Perceptron Recap
* A Perceptron is made up of TLU neurons
* TLUs take numerical input, multiply the inputs by weights, then sum the weighted inputs.
* TLUs then pass the sum through a step function, which returns binary output (either 0 or 1)
---

### Limitations of the Perceptron
* The Perceptron is not good for solving non-linearly seperable problems
* A small change in the weights within a TLU can cause its output to flip from 0 to 1 or 1 to 0.
* Perceptrons cannot solve XOR problems - multilayer perceptrons can though!
---

### Alternative to the TLU: The Sigmoid Neuron
* Similar to the TLU, but it doesn't use the step function
* Instead it uses the Sigmoid function. This means the output will be a float value between 0 and 1
* Small changes in the weights will cause small changes in the output meaning we don't have flipping!
---

In [63]:
#import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn import metrics

def check_NaN(dataframe):
    print("Total NaN:", dataframe.isnull().values.sum())
    print("NaN by column:\n",dataframe.isnull().sum())
    return

def one_hot_encode(dataframe, col_name):
    dataframe = pd.get_dummies(dataframe, columns=[col_name], prefix = [col_name], dtype= int)
    return dataframe

### Using a Perceptron to Classify Mushrooms as Edible or Poisonous
In this Notebook, we'll be using the mushroom classification dataset, which you can find here https://www.kaggle.com/uciml/mushroom-classification to train a Perceptron to determine whether a mushroom is edible (e) or poisonous (p), based its physical characteristics.

In [5]:
#load the dataset
data = pd.read_csv("./mushrooms.csv")

In [7]:
#check out its features
data.head()

class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
0            c         n          k  ...                        s   
1            c         b          k  ...                        s   
2            c         b          n  ...                        s   
3            c         n          n  ...                        s   
4            w         b          k  ...                        s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w         p          w   
1                      w                      w         p          w   
2                      w                      w         p          w   
3                      w                      w         p          w   
4                      w                      w         p          w   

  ring-number ring-type spore-print-color population habitat  
0           o         p                 k          s       u  
1           o         p                 n          n       g  
2           o         p                 n          n       m  
3           o         p                 k          s       u  
4           o         e                 n          a       g  

[5 rows x 23 columns]

Let's choose gill-size (narrow or broad) and spore print color as our features. Note spore-print-color: black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y

In [97]:
chosen_features = data.filter(['class','gill-size','spore-print-color', "cap-shape"])
chosen_features.head()

class gill-size spore-print-color cap-shape
0     p         n                 k         x
1     e         b                 n         x
2     e         b                 n         b
3     p         n                 k         x
4     e         b                 n         x

In [99]:
#always remember to check for NaN values
check_NaN(chosen_features)

Total NaN: 0
NaN by column:
 class                0
gill-size            0
spore-print-color    0
cap-shape            0
dtype: int64


One hot encode the chosen features

In [139]:
subset = one_hot_encode(chosen_features, 'class')
subset = one_hot_encode(subset, 'gill-size')
subset = one_hot_encode(subset, 'spore-print-color')
subset.head()

cap-shape  class_e  class_p  gill-size_b  gill-size_n  spore-print-color_b  \
0         x        0        1            0            1                    0   
1         x        1        0            1            0                    0   
2         b        1        0            1            0                    0   
3         x        0        1            0            1                    0   
4         x        1        0            1            0                    0   

   spore-print-color_h  spore-print-color_k  spore-print-color_n  \
0                    0                    1                    0   
1                    0                    0                    1   
2                    0                    0                    1   
3                    0                    1                    0   
4                    0                    0                    1   

   spore-print-color_o  spore-print-color_r  spore-print-color_u  \
0                    0                    0                    0   
1                    0                    0                    0   
2                    0                    0                    0   
3                    0                    0                    0   
4                    0                    0                    0   

   spore-print-color_w  spore-print-color_y  
0                    0                    0  
1                    0                    0  
2                    0                    0  
3                    0                    0  
4                    0                    0

Now, let's just pick the 'class_e' feature. This means if the Perceptron returns a value of 1, then the mushroom is edible. If it returns 0, then the mushroom is poisonous. Let's also pick 'gill_size_b', because the only other value it can be is 'gill_size_n', which means the gill size will be broad when it = 1, and narrow when it = 0. We'll pick all the colours to train on. 

In [142]:
final = subset.filter(['class_e','gill-size_b','spore-print-color_h','spore-print-color_h','spore-print-color_k','spore-print-color_n','spore-print-color_o','spore-print-color_r','spore-print-color_u','spore-print-color_w','spore-print-color_y'])
final.head()

class_e  gill-size_b  spore-print-color_h  spore-print-color_k  \
0        0            0                    0                    1   
1        1            1                    0                    0   
2        1            1                    0                    0   
3        0            0                    0                    1   
4        1            1                    0                    0   

   spore-print-color_n  spore-print-color_o  spore-print-color_r  \
0                    0                    0                    0   
1                    1                    0                    0   
2                    1                    0                    0   
3                    0                    0                    0   
4                    1                    0                    0   

   spore-print-color_u  spore-print-color_w  spore-print-color_y  
0                    0                    0                    0  
1                    0                    0                    0  
2                    0                    0                    0  
3                    0                    0                    0  
4                    0                    0                    0

In [144]:
#Create the train/test splits as we did before
x_train, x_test, y_train, y_test = train_test_split(final.drop(['class_e'], axis=1),final['class_e'],test_size=0.2,random_state=42)                                                                       
print("x train/test ",x_train.shape, x_test.shape)
print("y train/test ",y_train.shape, y_test.shape)

x train/test  (6499, 9) (1625, 9)
y train/test  (6499,) (1625,)


In [146]:
#Convert them from pandas to numpy arrays
x = x_train.values
y = y_train.values
x_t = x_test.values
y_t = y_test.values

In [148]:
x_t[0]

array([1, 0, 0, 1, 0, 0, 0, 0, 0])

In [150]:
#Create a perception model and train it
perceptron = Perceptron()
perceptron.fit(x, y)

Perceptron()

In [152]:
#run the model on the test set
predictions = perceptron.predict(x_t)
#Calculate the mean squared error and accuracy
print("Mean squared error: ",np.mean((predictions - y_t) ** 2))
print("Accuracy:",str(round(metrics.accuracy_score(y_t, predictions)*100))+"%")

Mean squared error:  0.04
Accuracy: 96%


Test a mushroom with a broad gill-size and black spore print color, where index = 0 is gill-size and index = 3 is black

In [155]:
test_mushroom = [1,0,0,0,0,0,0,1,0]
prediction = perceptron.predict([test_mushroom])

In [157]:
if prediction==1:
    print('Edible')
else:
    print('Poisonous')

Edible
